In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [8]:
df = con.execute("""
                 SELECT * FROM ( 
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-11-03'
                 ) WHERE row = 1
                 """).fetchdf()
df.head()

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-11-03 14:40:08.165141,1
1,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-11-03 14:40:08.165141,1
2,10004,SERRA,BT50,100,200,z0019_2.csv,2025-11-03 14:46:12.084505,1
3,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-11-03 14:46:12.084505,1
4,10003,PREGO,BT10,100,60,z0019_2.csv,2025-11-03 14:46:12.084505,1


In [10]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={'NATBR': 'id'})
df_final = df_final.rename(columns={'MAKTX': 'nm_produto'})
df_final = df_final.rename(columns={'WERKS': 'id_categoria'})
df_final = df_final.rename(columns={'MAINS': 'id_fornecedor'})
df_final = df_final.rename(columns={'LABST': 'vl_preco'})
df_final.head() 

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,10002,MARTELO,BT50,100,1500
2,10004,SERRA,BT50,100,200
3,10005,MACHADO,BT50,100,100
4,10003,PREGO,BT10,100,60


In [11]:
df2 = df_final.copy()
df2.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100
1,10002,MARTELO,BT50,100,1500
2,10004,SERRA,BT50,100,200
3,10005,MACHADO,BT50,100,100
4,10003,PREGO,BT10,100,60


In [12]:
df2.dtypes

id               object
nm_produto       object
id_categoria     object
id_fornecedor    object
vl_preco         object
dtype: object

In [14]:
df2 = df2.astype({
    'id': int,
    'nm_produto': str,
    'id_categoria': str,
    'id_fornecedor': int,
    'vl_preco': float
})
df2.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [15]:
con.execute("""
            CREATE TABLE IF NOT EXISTS produtos (
                id INTEGER,
                nm_produto VARCHAR,
                id_categoria VARCHAR,
                id_fornecedor INTEGER,
                vl_preco FLOAT
            )
""")

In [16]:
df2.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10002,MARTELO,BT50,100,1500.0
2,10004,SERRA,BT50,100,200.0
3,10005,MACHADO,BT50,100,100.0
4,10003,PREGO,BT10,100,60.0


In [19]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [20]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10001,PARAFUSO,BT10,100,100.0
1,10002,MARTELO,BT50,100,1500.0
2,10004,SERRA,BT50,100,200.0
3,10005,MACHADO,BT50,100,100.0
4,10003,PREGO,BT10,100,60.0


In [21]:
con.close()